In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import keras_tuner as kt
from a1 import build_deep_nn  
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input

In [7]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to [0, 1] range
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape data to include a channel dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [15]:
def build_model(hp):
    # Define the number of hidden layers
    num_hidden_layers = hp.Int('num_hidden_layers', 1, 3)
    
    # Define the size of the hidden layers
    hidden_size = hp.Int('hidden_size', 32, 256, step=32)
    
    # Define the dropout rate for the last layer
    dropout_rate = hp.Float('dropout_rate', 0.0, 0.5, step=0.1)
    
    # Create the layer options for build_deep_nn function
    layer_options = [(hidden_size, 'relu', 0.0) for _ in range(num_hidden_layers - 1)]
    layer_options.append((hidden_size, 'relu', dropout_rate))
    
    # Use the build_deep_nn function from a1.py
    model = build_deep_nn(28, 28, 1, layer_options)
    
    # Add the output layer
    model.add(Dense(10, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

# Set up the tuner
tuner = kt.BayesianOptimization(
    build_model,
    objective=kt.Objective('val_accuracy', direction='max'),
    max_trials=10,
    num_initial_points=2,
    overwrite=True
)

# Run the search
tuner.search(
    x_train, y_train, 
    validation_data=(x_test, y_test),
    epochs=5,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)]
)

Trial 5 Complete [00h 00m 34s]
val_accuracy: 0.9782000184059143

Best val_accuracy So Far: 0.9782000184059143
Total elapsed time: 00h 02m 39s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
1                 |1                 |num_hidden_layers
256               |192               |hidden_size
0                 |0                 |dropout_rate

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8887 - loss: 0.3835 - val_accuracy: 0.9656 - val_loss: 0.1140
Epoch 2/5
1033/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9698 - loss: 0.1046

KeyboardInterrupt: 

In [20]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters()[0]

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model 
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

# Print the best hyperparameters
print(f"Optimal number of hidden layers: {best_hps.get('num_hidden_layers')}")
print(f"Optimal hidden layer size: {best_hps.get('hidden_size')}")
print(f"Optimal dropout rate: {best_hps.get('dropout_rate')}")

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8864 - loss: 0.3993 - val_accuracy: 0.9655 - val_loss: 0.1173
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9681 - loss: 0.1087 - val_accuracy: 0.9714 - val_loss: 0.0905
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9802 - loss: 0.0651 - val_accuracy: 0.9758 - val_loss: 0.0790
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9857 - loss: 0.0471 - val_accuracy: 0.9751 - val_loss: 0.0773
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9899 - loss: 0.0338 - val_accuracy: 0.9759 - val_loss: 0.0787
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9731 - loss: 0.0955
Test accuracy: 0.9759
Optimal number of hidden layers: 1
Optimal hidden layer size: 192
Optimal dropout rate: 0.0


1. What are the hyperparameters of the optimal model?
   The optimal model was found by Keras Tuner with the following hyperparameters:
     -Number of Hidden Layers: The optimal model has 1 hidden layer.
     -Size of the Hidden Layer: The hidden layer has 192 neurons.
     -Dropout Rate of the Final Hidden Layer: The dropout rate for the final hidden layer is 0.0, which means no dropout was applied.
2. What are the accuracy results of the optimal model on the test set?
    -The accuracy of the optimal model on the test set is 97.59%. This indicates that the model correctly classified approximately 97.59% of the test        samples.